In [1]:
import planetary_computer
from pystac_client import Client

client = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1/",
    modifier=planetary_computer.sign_inplace,
)
asset = client.get_collection("sentinel-2-l2a").assets["geoparquet-items"]
asset

<Asset href=abfs://items/sentinel-2-l2a.parquet>

In [11]:
from adlfs import AzureBlobFileSystem

azure = AzureBlobFileSystem(**asset.extra_fields["table:storage_options"])
azure.get(
    "items/sentinel-2-l2a.parquet",
    "/Users/gadomski/Desktop/sentinel-2-l2a.parquet",
    recursive=True,
)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [4]:
from pathlib import Path
from typing import Any

import geopandas
import rustac
from tqdm.notebook import tqdm

TOP_LEVEL_ATTRIBUTES = [
    "type",
    "stac_extensions",
    "id",
    "geometry",
    "bbox",
    "links",
    "assets",
    "collection",
]
YEARS = [2024]
number_of_items = 0

def clean_dict(item: dict[str, Any]) -> dict[str, Any]:
    for key, value in item.items():
        if hasattr(value, "tolist"):
            item[key] = value.tolist()
        elif hasattr(value, "isoformat"):
            item[key] = value.isoformat()
        elif isinstance(value, dict):
            item[key] = clean_dict(value)
    return item

Path("/Users/gadomski/Desktop/sentinel-2-l2a-fixed.parquet").mkdir(exist_ok=True)

for path in Path("/Users/gadomski/Desktop/sentinel-2-l2a.parquet").glob("*.parquet"):
    if int(path.stem.split("_")[1].split("-")[0]) in YEARS:
        data_frame = geopandas.read_parquet(path)
        feature_collection = data_frame.to_geo_dict()
        items = []
        for feature in tqdm(feature_collection["features"]):
            item = clean_dict(feature["properties"])
            new_item = {
                "geometry": feature["geometry"]
            }
            properties = {}
            for key, value in item.items():
                if key in TOP_LEVEL_ATTRIBUTES:
                    new_item[key] = value
                else:
                    properties[key] = value
            new_item["properties"] = properties
            items.append(new_item)
        number_of_items += len(items)
        await rustac.write(
            str(
                Path("/Users/gadomski/Desktop/sentinel-2-l2a-fixed.parquet") / path.name
            ),
            items,
        )
        print(number_of_items, "items")


  0%|          | 0/79282 [00:00<?, ?it/s]

79282 items


  0%|          | 0/79608 [00:00<?, ?it/s]

158890 items


  0%|          | 0/67992 [00:00<?, ?it/s]

...

2041868 items


  0%|          | 0/74488 [00:00<?, ?it/s]

2116356 items


  0%|          | 0/79131 [00:00<?, ?it/s]

2195487 items
